In [ ]:
# !python --version
# !python -m pip install graphviz

# Lexer File

It's used to tokenize the input regex string

> Note: Lexer completely ignores escape character,
> so whatever character you choose for #them, it’s one you can’t match literally
> (and you can easily modify it to allow escaping #the escapes themselves)

In [ ]:
from enum import Enum, auto


# | * + ? ( ) [ ] - are the meta characters
class TokenType(Enum):
    OR = auto()
    STAR = auto()
    PLUS = auto()
    DASH = auto()
    LITERAL_CHARACTER = auto()
    QUESTION_MARK = auto()
    OPEN_PARENTHESIS = auto()
    CLOSED_PARENTHESIS = auto()
    OPEN_SQUARE_BRACKET = auto()
    CLOSED_SQUARE_BRACKET = auto()


class Token:
    # colab error: unsupported operand type(s) for |: 'type' and 'NoneType'
    # def __init__(self, token_type: TokenType, value: str | None):
    def __init__(self, token_type: TokenType, value):
        self.token_type = token_type
        self.value = value

    def __str__(self):
        return f"<{self.token_type}, {self.value}>"

    def __repr__(self):
        return f"<{self.token_type}, {self.value}>"


class Lexer:
    Escape_Character = "/"
    Meta_Characters_Map = {
        "|": TokenType.OR,
        "*": TokenType.STAR,
        "+": TokenType.PLUS,
        "?": TokenType.QUESTION_MARK,
        "(": TokenType.OPEN_PARENTHESIS,
        ")": TokenType.CLOSED_PARENTHESIS,
        "[": TokenType.OPEN_SQUARE_BRACKET,
        "]": TokenType.CLOSED_SQUARE_BRACKET,
        "-": TokenType.DASH,
    }

    def __init__(self, input_regex: str):
        self.input_regex = input_regex
        self.tokens = []

    def tokenize(self) -> list[Token]:
        prev_char = None
        for char in self.input_regex:
            if char == Lexer.Escape_Character:
                prev_char = char
                continue
            if char in Lexer.Meta_Characters_Map and prev_char != Lexer.Escape_Character:
                self.tokens.append(Token(Lexer.Meta_Characters_Map[char], char))
            else:
                self.tokens.append(Token(TokenType.LITERAL_CHARACTER, char))
            prev_char = char
        return self.tokens
        

# AST Classes

This `Parser` will be using those classes when parsing the tokens returned from the `Lexer`

In [ ]:
class AstNode:
    pass


class OrAstNode(AstNode):
    def __init__(self, left: AstNode, right: AstNode):
        self.left = left
        self.right = right

    def __str__(self):
        return f"({self.left} | {self.right})"

    def __repr__(self):
        return f"({self.left} | {self.right})"


class SeqAstNode(AstNode):
    def __init__(self, left: AstNode, right: AstNode):
        self.left = left
        self.right = right

    def __str__(self):
        return f"({self.left} {self.right})"

    def __repr__(self):
        return f"({self.left} {self.right})"


class StarAstNode(AstNode):
    def __init__(self, left: AstNode):
        self.left = left

    def __str__(self):
        return f"({self.left}*)"

    def __repr__(self):
        return f"({self.left}*)"


class PlusAstNode(AstNode):
    def __init__(self, left: AstNode):
        self.left = left

    def __str__(self):
        return f"({self.left}+)"

    def __repr__(self):
        return f"({self.left}+)"


class QuestionMarkAstNode(AstNode):
    def __init__(self, left: AstNode):
        self.left = left

    def __str__(self):
        return f"({self.left}?)"

    def __repr__(self):
        return f"({self.left}?)"


class LiteralCharacterAstNode(AstNode):
    def __init__(self, char: str):
        self.char = char

    def __str__(self):
        return self.char

    def __repr__(self):
        return self.char


class CharacterClassAstNode(AstNode):
    # colab error: unsupported operand type(s) for |: 'type' and 'types.GenericAlias'
    # def __init__(self, char_class: set[str | tuple[str, str]]): 
    def __init__(self, char_class):
        self.char_class = char_class

    def __str__(self):
        return f"[{self.char_class}]"

    def __repr__(self):
        return f"[{self.char_class}]"

# Parser File

This will build the necessary `AST` so later the compiler backend will be able to use it and build the `NFA`, `DFA` and `Minmized DFA` 


### The Grammer used as the guide line for the parser is as following:
```
regex-expr = regex-or-expr

regex-or-expr = regex-seq-expr (OR_TOKEN regex-seq-expr)*

regex-seq-expr = regex-quantified-expr (regex-quantified-expr)*

regex-quantified-expr = 
  regex-base-expr (STAR_TOKEN | PLUS_TOKEN | QUESTION_MARK_TOKEN)?

regex-base-expr ::= 
  LITERAL_CHAR_TOKEN | 
  OPEN_SQUARE_BRACKET_TOKEN square-bracket-content CLOSED_SQUARE_BRACKET_TOKEN |
  OPEN_PARENTHESIS_TOKEN regex-expr CLOSED_PARENTHESIS_TOKEN

square-bracket-content = square-bracket-element+

square-bracket-element = 
  LITERAL_CHAR_TOKEN | 
  LITERAL_CHAR_TOKEN DASH LITERAL_CHAR_TOKEN

```

In [ ]:
from typing import Tuple


class Parser:
    def __init__(self, tokens: list[Token]):
        self.tokens = tokens

    def parse(self) -> AstNode:
        return self.__parse(0)[0]

    def __parse(self, index: int) -> Tuple[AstNode, int]:
        return self.__parse_or(index)

    def __parse_or(self, index: int) -> Tuple[AstNode, int]:
        left, index = self.__parse_seq(index)
        rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            return (left, index)
        result = left
        while rem_tokens and self.tokens[index].token_type == TokenType.OR:
            index += 1
            right, index = self.__parse_seq(index)
            result = OrAstNode(result, right)
            rem_tokens = index < len(self.tokens)
        return (result, index)

    def __parse_seq(self, index: int) -> Tuple[AstNode, int]:
        left, index = self.__parse_counters(index)
        rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            return (left, index)
        result = left
        while (
            rem_tokens
            and self.tokens[index].token_type != TokenType.OR
            and self.tokens[index].token_type != TokenType.CLOSED_PARENTHESIS
        ):
            right, index = self.__parse_counters(index)
            result = SeqAstNode(result, right)
            rem_tokens = index < len(self.tokens)
        return (result, index)

    def __parse_counters(self, index: int) -> Tuple[AstNode, int]:
        left, index = self.__parse_base(index)
        rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            return (left, index)
        # if not rem_tokens:
        #     return (left, index)
        # if rem_tokens and self.tokens[index].token_type == TokenType.QUESTION_MARK:
        #     index += 1
        #     return (QuestionMarkAstNode(left), index)
        # if rem_tokens and self.tokens[index].token_type == TokenType.STAR:
        #     index += 1
        #     return (StarAstNode(left), index)
        # if rem_tokens and self.tokens[index].token_type == TokenType.PLUS:
        #     index += 1
        #     return (PlusAstNode(left), index)
        while rem_tokens and self.tokens[index].token_type in [TokenType.PLUS, TokenType.STAR, TokenType.QUESTION_MARK]:
            if rem_tokens and self.tokens[index].token_type == TokenType.QUESTION_MARK:
                index += 1
                left = QuestionMarkAstNode(left)
            elif rem_tokens and self.tokens[index].token_type == TokenType.STAR:
                index += 1
                left = StarAstNode(left)
            elif rem_tokens and self.tokens[index].token_type == TokenType.PLUS:
                index += 1
                left = PlusAstNode(left)
            rem_tokens = index < len(self.tokens)
        return (left, index)

    def __parse_square_bracket(self, index: int) -> Tuple[AstNode, int]:
        rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            raise Exception()
        chars = []
        end_range = False
        while rem_tokens and self.tokens[index].token_type != TokenType.CLOSED_SQUARE_BRACKET:
            if self.tokens[index].token_type == TokenType.DASH:
                end_range = True
            elif end_range:
                if len(chars) == 0:
                    raise Exception()
                start = chars.pop()
                end = self.tokens[index].value
                if start > end:
                    raise Exception()
                chars.append((start, end))
                end_range = False
            else:
                chars.append(self.tokens[index].value)
            index += 1
            rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            raise Exception()
        return (CharacterClassAstNode(set(chars)), index)

    def __parse_base(self, index: int) -> Tuple[AstNode, int]:
        rem_tokens = index < len(self.tokens)
        if not rem_tokens:
            raise Exception()
        current_token = self.tokens[index]
        index += 1
        if current_token.token_type == TokenType.LITERAL_CHARACTER:
            return (LiteralCharacterAstNode(current_token.value), index)
        if current_token.token_type == TokenType.OPEN_PARENTHESIS:
            left, index = self.__parse(index)
            rem_tokens = index < len(self.tokens)
            if not rem_tokens:
                raise Exception()
            if self.tokens[index].token_type != TokenType.CLOSED_PARENTHESIS:
                raise Exception()
            index += 1
            return (left, index)
        if current_token.token_type == TokenType.OPEN_SQUARE_BRACKET:
            left, index = self.__parse_square_bracket(index)
            rem_tokens = index < len(self.tokens)
            if not rem_tokens:
                raise Exception()
            if self.tokens[index].token_type != TokenType.CLOSED_SQUARE_BRACKET:
                raise Exception()
            index += 1
            return (left, index)

# NFA



In [ ]:
from enum import Enum
from typing import Dict, List, Set, Tuple


class State:
    def __init__(self, label: str, accept: bool = False):
        self.label = label
        self.accept = accept

    def __str__(self):
        return self.label

    def __repr__(self):
        return self.label

    def __eq__(self, other):
        if isinstance(other, State):
            return self.label == other.label
        return False

    def __lt__(self, other):
        if isinstance(other, State):
            return self.label < other.label
        return False

    def __hash__(self):
        return hash(self.label)


class ThompsonNFA:
    def __init__(self, start: State, end: State):
        self.start = start
        self.end = end


EPSILON = "ε"


# transition table will be like this:
# {from_state: [(to_state, char)]}
# where from_state is the state that the transition is coming from
# and to_state is the state that the transition is going to
# and char is the character that the transition is on the edge & epsilons
__transition_table: Dict[State, List[Tuple[State, str]]] = {}

# to store the initial and terminal states of the NFA
__starting_state, __accepting_state = State(None), State(None)

# set the verbosity level of the NFA
# i.e whether to expand ranges like [a-z] to [a, b, c, ..., z] or not
__verbose = False

def get_transition_table() -> Dict[State, List[Tuple[State, str]]]:
    return __transition_table


def get_starting_state() -> State:
    global __starting_state
    return __starting_state


def get_accepting_state() -> State:
    global __accepting_state
    return __accepting_state


def ast_to_nfa(root: AstNode, verbose: bool = False) -> None:
    global __verbose
    __verbose = verbose
    _, _ = __ast_to_nfa(root=root, index=0)
    # __transition_table[nfa.start] = [(nfa.end, EPSILON)]


def __add_transition(from_state: State, to_state: State, char: str) -> None:
    global __transition_table
    if from_state not in __transition_table:
        __transition_table[from_state] = []
    # print(f"adding transition from {from_state} to {to_state} on {char}")
    __transition_table[from_state].append((to_state, char))


def __update_start_finish(start: State, end: State) -> None:
    global __starting_state, __accepting_state
    __starting_state = start
    __accepting_state = end


def __ast_to_nfa(root: AstNode, index: int = 0) -> Tuple[ThompsonNFA, int]:
    global __verbose
    if root is None:
        start = State(f"S{index}")
        end = State(f"S{index + 1}")
        __update_start_finish(start, end)
        # __add_transition(start, end, EPSILON)
        return ThompsonNFA(start, end), index + 2
    if isinstance(root, LiteralCharacterAstNode):
        return __literal_character_ast_to_nfa(root.char, index)
    if isinstance(root, OrAstNode):
        return __or_ast_to_nfa(root, index)
    if isinstance(root, SeqAstNode):
        return __seq_ast_to_nfa(root, index)
    if isinstance(root, StarAstNode):
        return __star_ast_to_nfa(root, index)
    if isinstance(root, PlusAstNode):
        return __plus_ast_to_nfa(root, index)
    if isinstance(root, QuestionMarkAstNode):
        return __question_mark_ast_to_nfa(root, index)
    if isinstance(root, CharacterClassAstNode):
        if __verbose:
            return __character_class_ast_to_nfa_verbose(root, index)
        else:
            return __character_class_ast_to_nfa(root, index)


def __literal_character_ast_to_nfa(root_char: str, index: int) -> Tuple[ThompsonNFA, int]:
    start = State(f"S{index}")
    end = State(f"S{index + 1}")
    __add_transition(start, end, root_char)
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 2


def __or_ast_to_nfa(root: OrAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
          -e-> S2 -a-> S3 -e->
         //                   \\
     -> S0                     -> S6
         \\                   //
          -e-> S4 -b-> S5 -e->
    """
    start = State(f"S{index}")  # S0
    left_nfa, index = __ast_to_nfa(root.left, index + 1)
    right_nfa, index = __ast_to_nfa(root.right, index + 1)
    end = State(f"S{index}")  # S6
    __add_transition(start, left_nfa.start, EPSILON)  # S0 -e-> S2
    __add_transition(start, right_nfa.start, EPSILON)  # S0 -e-> S4
    __add_transition(left_nfa.end, end, EPSILON)  # S3 -e-> S6
    __add_transition(right_nfa.end, end, EPSILON)  # S5 -e-> S6
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 1


def __seq_ast_to_nfa(root: SeqAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
    -> S0 -a-> S1 -e-> S2 -b-> S3
    """
    start = State(f"S{index}")  # S0
    left_nfa, index = __ast_to_nfa(root.left, index + 1)
    right_nfa, index = __ast_to_nfa(root.right, index + 1)
    __add_transition(start, left_nfa.start, EPSILON)  # S0 -e-> S1
    __add_transition(left_nfa.end, right_nfa.start, EPSILON)  # S2 -e-> S3
    __update_start_finish(start, right_nfa.end)
    return ThompsonNFA(start, right_nfa.end), index + 1


def __star_ast_to_nfa(root: StarAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
        v------e-------|
    -> S0 -e-> S1 -a-> S2 -e-> S3
        ^---------e------------|
    """
    start = State(f"S{index}")  # S0
    nfa, index = __ast_to_nfa(root.left, index + 1)
    end = State(f"S{index}")  # S3
    __add_transition(start, end, EPSILON)  # S0 -e-> S3
    __add_transition(start, nfa.start, EPSILON)  # S0 -e-> S1
    __add_transition(nfa.end, end, EPSILON)  # S2 -e-> S3
    __add_transition(nfa.end, nfa.start, EPSILON)  # S2 -e-> S1
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 1


def __plus_ast_to_nfa(root: PlusAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
        v------e-------|
    -> S0 -e-> S1 -a-> S2 -e-> S3
    """
    start = State(f"S{index}")  # S0
    nfa, index = __ast_to_nfa(root.left, index + 1)
    end = State(f"S{index}")  # S3
    __add_transition(start, nfa.start, EPSILON)  # S0 -e-> S1
    __add_transition(nfa.end, end, EPSILON)  # S2 -e-> S3
    __add_transition(nfa.end, nfa.start, EPSILON)  # S2 -e-> S1
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 1


def __question_mark_ast_to_nfa(root: QuestionMarkAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
        |------e-------v
    -> S0 -e-> S1 -a-> S2
                |--e---^
    """
    start = State(f"S{index}")  # S0
    nfa, index = __ast_to_nfa(root.left, index + 1)
    end = State(f"S{index}")  # S2
    __add_transition(start, end, EPSILON)  # S0 -e-> S2
    __add_transition(start, nfa.start, EPSILON)  # S0 -e-> S1
    __add_transition(nfa.end, end, EPSILON)  # S2 -e-> S2
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 1


def __character_class_ast_to_nfa(root: CharacterClassAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
    this time root it has a set[str | Tuple[str, str]] so in
    1. just a char: it's just a literal char
    2. a range: it's a range of chars like [a-z] => "a-z" for simplicity
        2.a it could be written as [abc...z], later maybe !
        2.b in case of reversed range => the parser will throw an error
    then consider all of the result as ored literals
    """
    start = State(f"S{index}")  # S0
    nfas = []
    for char in root.char_class:
        if isinstance(char, str):
            value = char
        else:
            value = f"{char[0]}-{char[1]}"
        nfa, index = __literal_character_ast_to_nfa(value, index + 1)
        nfas.append(nfa)

    # or each 2 nfas together
    while len(nfas) > 1:
        nfa1 = nfas.pop()
        nfa2 = nfas.pop()
        semi_start = State(f"S{index + 1}")  # S0
        semi_end = State(f"S{index + 2}")  # S1
        __add_transition(semi_start, nfa1.start, EPSILON)  # S0 -e-> S1
        __add_transition(semi_start, nfa2.start, EPSILON)  # S0 -e-> S2
        __add_transition(nfa1.end, semi_end, EPSILON)  # S3 -e-> S4
        __add_transition(nfa2.end, semi_end, EPSILON)  # S5 -e-> S4
        nfas.append(ThompsonNFA(semi_start, semi_end))
        index += 2

    end = State(f"S{index + 1}")  # S1
    __add_transition(nfas[0].end, end, EPSILON)  # S3 -e-> S4
    __add_transition(start, nfas[0].start, EPSILON)  # S0 -e-> S1
    __update_start_finish(start, end)
    return ThompsonNFA(start, end), index + 2

# def __character_class_ast_to_nfa_verbose(root: CharacterClassAstNode, index: int) -> Tuple[ThompsonNFA, int]:
# """
#                     -e-> S2 -a-> S3 -e->
#                   //                    \\
#               -> S0                     S6 ->
#             //    \\                    //   \\
#            //      -e-> S4 -b-> S5 -e-->      \\
#        -> S0                                   S7 ->
#            \\                                 //
#             \\                               //
#              ---e----> S4 -b-> S5 -----e----->
# """
#     def build_or_ast_from_set(chars: Set[str]) -> AstNode:
#         if len(chars) == 1:
#             return LiteralCharacterAstNode(chars.pop())
#         else:
#             char = chars.pop()
#             return OrAstNode(LiteralCharacterAstNode(char), build_or_ast_from_set(chars))
#     all_chars = set()
#     for char in root.char_class:
#         if isinstance(char, str):
#             all_chars.add(char)
#         else:
#             for c in range(ord(char[0]), ord(char[1]) + 1):
#                 all_chars.add(chr(c))
#     return __ast_to_nfa(build_or_ast_from_set(all_chars), index)


def __character_class_ast_to_nfa_verbose(root: CharacterClassAstNode, index: int) -> Tuple[ThompsonNFA, int]:
    """
          ---------a-------->
         //                 \\
     -> S0 ---------b-------> S1
         \\                 //
          ---------c-------->
    """
    start = State(f"S{index}")  # S0
    index += 1
    end = State(f"S{index}")

    all_chars = set()
    for char in root.char_class:
        if isinstance(char, str):
            all_chars.add(char)
        else:
            for c in range(ord(char[0]), ord(char[1]) + 1):
                all_chars.add(chr(c))

    for char in all_chars:
        __add_transition(start, end, char)

    __update_start_finish(start, end)

    return ThompsonNFA(start, end), index + 1



# DFA

In [ ]:
# this file is used to generate the DFA from the NFA

from typing import Dict, List, Tuple, Set
from copy import deepcopy


class DFA:
    def __init__(
        self,
        starting_state: frozenset[State],
        accepting_states: list[frozenset[State]],
        transitions: Dict[frozenset[State], Set[Tuple[frozenset[State], str]]],
        all_states: Set[frozenset[State]],
    ):
        self.starting_state = starting_state
        self.accepting_states = accepting_states
        self.transitions = transitions
        self.all_states = all_states


class DFAClean:
    def __init__(
        self,
        starting_state: State,
        accepting_states: list[State],
        transitions: Dict[State, Set[Tuple[State, str]]],
        all_states: Set[State],
    ):
        self.starting_state = starting_state
        self.accepting_states = accepting_states
        self.transitions = transitions
        self.all_states = all_states


def __get_epsilon_closure(nfa_start: State, nfa_transitions: Dict[State, List[Tuple[State, str]]]) -> Set[State]:
    """
    Returns the epsilon closure of the given NFA start state.

    Args:
        nfa_start: The start state of the NFA.
        nfa_transitions: The transitions of the NFA. it's in the form of:
        {from_state: [(to_state, transition_symbol|ε), ...], }

    Returns:
        The epsilon closure of the given NFA start state.
    """

    # A state R is in the epsilon closure of a state S if
    # 1- S is R
    # 2- R can be reached via an epsilon transition from any state in the epsilon closure of S
    epsilon_closure = set()
    epsilon_closure.add(nfa_start)
    still_adding = True
    while still_adding:
        still_adding = False
        for state in deepcopy(epsilon_closure):
            for next_state, char in nfa_transitions.get(state, []):
                if char == EPSILON and next_state not in epsilon_closure:
                    still_adding = True
                    epsilon_closure.add(next_state)
    return epsilon_closure


def build_powerset(
    nfa_start: State,
    nfa_accepting: State,
    nfa_transitions: Dict[State, List[Tuple[State, str]]],
) -> DFA:
    """
    Builds the powerset of the given NFA.

    Args:
        nfa_start: The start state of the NFA.
        nfa_accepting: The accepting state of the NFA.
        nfa_transitions: The transitions of the NFA. it's in the form of:
        {from_state: [(to_state, transition_symbol|ε), ...], }

    Returns:
        a DFA {starting_state, accepting_states, transitions, all_states}
    """

    dfa_start = __get_epsilon_closure(nfa_start, nfa_transitions)
    dfa_accept: List[frozenset[State]] = []
    dfa_states: Set[frozenset[State]] = set()
    dfa_transitions: Dict[frozenset[State], Set[Tuple[frozenset[State], str]]] = {}

    superstates_to_process: List[Set[State]] = [dfa_start]

    while superstates_to_process:
        superstate = superstates_to_process.pop()
        frozen_superstate = frozenset(superstate)
        dfa_states.add(frozen_superstate)
        if nfa_accepting in superstate:
            dfa_accept.append(frozen_superstate)

        superstate_transitions: Dict[str, Set[State]] = {}
        for state in superstate:
            for next_state, char in nfa_transitions.get(state, []):
                if char != EPSILON:
                    state_moving = __get_epsilon_closure(next_state, nfa_transitions)
                    if char in superstate_transitions:
                        superstate_transitions[char] = superstate_transitions[char].union(state_moving)
                    else:
                        superstate_transitions[char] = state_moving

        for char, next_superstate in superstate_transitions.items():
            frozen_next_superstate = frozenset(next_superstate)
            if frozen_superstate in dfa_transitions:
                dfa_transitions[frozen_superstate].add((frozen_next_superstate, char))
            else:
                dfa_transitions[frozen_superstate] = {(frozen_next_superstate, char)}
            if next_superstate not in dfa_states:
                superstates_to_process.append(next_superstate)

    return DFA(frozenset(dfa_start), dfa_accept, dfa_transitions, dfa_states)


def clean_dfa(dfa: DFA) -> DFAClean:
    """
    Cleans the given DFA i.e exchange the supersets with just a single state representing them.

    Args:
        dfa: The DFA to clean.

    Returns:
        A DFAClean {starting_state, accepting_states, transitions, all_states}
    """
    index = 0
    superstate_to_state: Dict[frozenset[State], State] = {}
    for superstate in dfa.all_states:
        superstate_to_state[superstate] = State(f"S{index}")
        index += 1
    clean_start = superstate_to_state[dfa.starting_state]
    clean_accepting = [superstate_to_state[superstate] for superstate in dfa.accepting_states]

    clean_transitions: Dict[State, Set[Tuple[State, str]]] = {}
    for superstate, transitions in dfa.transitions.items():
        for next_superstate, char in transitions:
            if superstate_to_state[superstate] in clean_transitions:
                clean_transitions[superstate_to_state[superstate]].add((superstate_to_state[next_superstate], char))
            else:
                clean_transitions[superstate_to_state[superstate]] = {(superstate_to_state[next_superstate], char)}

    clean_all_states = set(superstate_to_state.values())

    return DFAClean(clean_start, clean_accepting, clean_transitions, clean_all_states)


# Minimzed DFA

In [ ]:

from typing import Dict, List, Tuple, Set
from copy import deepcopy


class Hashabledict(dict):
    def __hash__(self):
        return hash(frozenset(self.items()))


def minimize_dfa(cdfa: DFAClean) -> DFAClean:
    accepting_states = set(cdfa.accepting_states)
    rejecting_states = cdfa.all_states - accepting_states
    accepting_states = frozenset(accepting_states)
    rejecting_states = frozenset(rejecting_states)

    which_group = {}
    for state in cdfa.all_states:
        if state not in accepting_states:
            which_group[state] = rejecting_states
        else:
            which_group[state] = accepting_states

    mdfa_all_groups = {accepting_states, rejecting_states}
    still_splitting = True

    while still_splitting:
        still_splitting = False
        mdfa_all_groups_copy = deepcopy(mdfa_all_groups)
        for group in mdfa_all_groups:
            if len(group) > 1:
                possible_splits = {}
                for state in group:
                    state_transitions = Hashabledict()
                    for next_state, char in cdfa.transitions.get(state, []):
                        state_transitions[char] = which_group[next_state]

                    if state_transitions not in possible_splits:
                        possible_splits[state_transitions] = {state}
                    else:
                        possible_splits[state_transitions].add(state)

                if len(possible_splits) > 1:
                    still_splitting = True
                    mdfa_all_groups_copy.remove(group)
                    for new_group in possible_splits.values():
                        new_group = frozenset(new_group)
                        mdfa_all_groups_copy.add(new_group)
                        for state in new_group:
                            which_group[state] = new_group

                mdfa_all_groups = deepcopy(mdfa_all_groups_copy)

    mdfa_starting_state = which_group[cdfa.starting_state]

    mdfa_accepting_states = []
    for state in cdfa.accepting_states:
        mdfa_accepting_states.append(which_group[state])

    mdfa_transitions = {}
    for state, transitions in cdfa.transitions.items():
        mdfa_transitions[which_group[state]] = []
        for next_state, char in transitions:
            mdfa_transitions[which_group[state]].append((which_group[next_state], char))

    mdfa_all_states = set()
    for group in mdfa_all_groups:
        mdfa_all_states.add(frozenset(group))

    intermediate = DFA(mdfa_starting_state, mdfa_accepting_states, mdfa_transitions, mdfa_all_states)
    return clean_dfa(intermediate)


# Logger Helper functions

In [ ]:
# this file will be used to log down the nfa, dfa and mdfa
# into the json file with the desired format which is like this:
"""
{
    "startingState": "S0"
    "S0": {
        isTerminatingState: true,
        "0": "S0",
        "1": "S1"
    },
    "S1": {
        isTerminatingState: false,
        "0": "S2",
        "1": "S0"
    },
    "S2": {
        isTerminatingState: true,
        "0": "S1",
        "1": "S2"
    }
}
"""

import json
from typing import Dict, List, Tuple


__nfa_filename = "NFA.json"
# colab error: unsupported operand type(s) for |: 'type' and 'type'
# __nfa_result: Dict[str, str | Dict[str, str | bool]] = {}
__nfa_result = {}

__mdfa_filename = "MDFA.json"
# colab error: unsupported operand type(s) for |: 'type' and 'type'
# __mdfa_result: Dict[str, str | Dict[str, str | bool]] = {}
__mdfa_result = {}


def __build_nfa_result(transition_table, accepting_state: State):
    accepted_labeled = False
    for state, transitions in transition_table.items():
        __nfa_result[state.label] = {}
        accepted_labeled = accepted_labeled or (state == accepting_state)
        for next_state, char in transitions:
            # if the char has been added before, then make it a list and append the next state
            if char in __nfa_result[state.label]:
                if isinstance(__nfa_result[state.label][char], list):
                    __nfa_result[state.label][char].append(next_state.label)
                else:
                    __nfa_result[state.label][char] = [__nfa_result[state.label][char], next_state.label]
            else:
                __nfa_result[state.label][char] = next_state.label
        __nfa_result[state.label]["isTerminatingState"] = state == accepting_state
    if not accepted_labeled:
        __nfa_result[accepting_state.label] = {"isTerminatingState": True}


def log_nfa(transition_table: Dict[State, List[Tuple[State, str]]], starting_state: State, accepting_state: State):
    __nfa_result.update({"startingState": starting_state.label})
    __build_nfa_result(transition_table, accepting_state)
    with open(__nfa_filename, "w") as f:
        json.dump(__nfa_result, f, ensure_ascii=False, indent=2)  # ensure utf-8 encoding


def __build_mdfa_result(mdfa: DFAClean):
    for state, transitions in mdfa.transitions.items():
        __mdfa_result[state.label] = {}
        for next_state, char in transitions:
            __mdfa_result[state.label][char] = next_state.label
        __mdfa_result[state.label]["isTerminatingState"] = state in mdfa.accepting_states


def log_mdfa(mdfa: DFAClean):
    __mdfa_result.update({"startingState": mdfa.starting_state.label})
    __build_mdfa_result(mdfa)
    with open(__mdfa_filename, "w") as f:
        json.dump(__mdfa_result, f, ensure_ascii=False, indent=2)


# Graph and Visualize the Reults

In [ ]:
import graphviz
from typing import Dict, List, Tuple


def visualize_nfa(
    transition_table: Dict[State, List[Tuple[State, str]]],
    starting_state: State,
    accepting_state: State,
    filename: str = "NFA",
):
    g = graphviz.Digraph("NFA", filename=filename, format="png")
    # make the graph horizontal
    g.attr(rankdir="LR")
    g.edge("", starting_state.label)  # add an arrow entering the starting state
    g.node("", shape="none")  # and remove the very first circle
    for state, transitions in transition_table.items():
        for next_state, char in transitions:
            g.edge(state.label, next_state.label, label=char)
    # add another oval for the accepting state
    g.node(accepting_state.label, peripheries="2")
    # add a title two lines under the graph
    g.attr(label=r"\n\nNFA", fontsize="20", labelloc="b")
    g.view()


def __frozenset_str(frozenset: frozenset[State]) -> str:
    """
    Returns the name of the given frozenset like in set.__str__
    """
    return "{" + ", ".join([state.label for state in frozenset]) + "}"


def visualize_dfa(dfa: DFA, filename: str = "DFA"):
    g = graphviz.Digraph("DFA", filename=filename, format="png")
    g.attr(rankdir="LR")
    g.edge("", __frozenset_str(dfa.starting_state))
    g.node("", shape="none")
    for state, transitions in dfa.transitions.items():
        x = __frozenset_str(state)
        for next_state, char in transitions:
            y = __frozenset_str(next_state)
            g.edge(x, y, label=char)
    for accepting_state in dfa.accepting_states:
        g.node(__frozenset_str(accepting_state), peripheries="2")
    g.attr(label=r"\n\nDFA", fontsize="20", labelloc="b")
    g.view()


def visualize_clean_dfa(clean_dfa: DFAClean, filename: str = "DFAClean"):
    g = graphviz.Digraph("DFAClean", filename=filename, format="png")
    g.attr(rankdir="LR")
    g.edge("", clean_dfa.starting_state.label)
    g.node("", shape="none")
    for state, transitions in clean_dfa.transitions.items():
        for next_state, char in transitions:
            g.edge(state.label, next_state.label, label=char)
    for accepting_state in clean_dfa.accepting_states:
        g.node(accepting_state.label, peripheries="2")
    g.attr(label=r"\n\nDFA Clean", fontsize="20", labelloc="b")
    g.view()


def visualize_mdfa(mdfa: DFAClean, filename: str = "MDFA"):
    g = graphviz.Digraph("MDFA", filename=filename, format="png")
    g.attr(rankdir="LR")
    g.edge("", mdfa.starting_state.label)
    g.node("", shape="none")
    for state, transitions in mdfa.transitions.items():
        for next_state, char in transitions:
            g.edge(state.label, next_state.label, label=char)
    for accepting_state in mdfa.accepting_states:
        g.node(accepting_state.label, peripheries="2")
    g.attr(label=r"\n\nMinimized DFA", fontsize="20", labelloc="b")
    g.view()


# Main

In [ ]:
def get_args():
    parser = argparse.ArgumentParser(description="A simple regex compiler that compiles a regex to a NFA")
    parser.add_argument(
        "regex",
        type=str,
        help="the regex to compile",
    )
    # make the verbose flag optional when given set it to true
    parser.add_argument(
        "-v",
        "--verbose",
        action="store_true",
        help="expand ranges like [a-z] to [a, b, c, ..., z] not just [a-z] on one edge",
    )
    return parser.parse_args()


def run(input_regex: str, verbose: bool = False):
    lexer = Lexer(input_regex)
    tokens = lexer.tokenize()

    parser = Parser(tokens)
    ast = parser.parse()
    print(tokens)
    print(ast)

    ast_to_nfa(ast, verbose=verbose)
    nfa = get_transition_table()

    starting_state = get_starting_state()
    accepting_state = get_accepting_state()
    log_nfa(nfa, starting_state, accepting_state)
    visualize_nfa(nfa, starting_state, accepting_state)

    dfa = build_powerset(starting_state, accepting_state, nfa)
    visualize_dfa(dfa)

    cdfa = clean_dfa(dfa)
    visualize_clean_dfa(cdfa)

    mdfa = minimize_dfa(cdfa)
    log_mdfa(mdfa)
    visualize_mdfa(mdfa, "MDFA")


# Testing

In [ ]:
regex = [
"(AB)", # 0
"(A|B)", # 1
"([A-Z])", # 2
"(A+)", # 3
"(A*)", # 4
"(((AB)((A|B)*))(AB))", # 5
"((((AB)|[A-Z])+)([A-Z]*))", # 6
"(((((ABE)|C)|((([A-Z])S)*))+)((AB)C))", # 7
"((([a-z_])(([a-z0-9_])*))(([!?])?))", # 8
"(A(((B*)|(DA))*))((CG)|(D([DEF])))", # 9
"(ab", # 10
"(a([b-c))", # 11
"((a|b)|)", # 12
"(a{3,2})" # 13
]

run(regex[8], verbose=True)

[<TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_SQUARE_BRACKET, [>, <TokenType.LITERAL_CHARACTER, a>, <TokenType.DASH, ->, <TokenType.LITERAL_CHARACTER, z>, <TokenType.LITERAL_CHARACTER, _>, <TokenType.CLOSED_SQUARE_BRACKET, ]>, <TokenType.CLOSED_PARENTHESIS, )>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_SQUARE_BRACKET, [>, <TokenType.LITERAL_CHARACTER, a>, <TokenType.DASH, ->, <TokenType.LITERAL_CHARACTER, z>, <TokenType.LITERAL_CHARACTER, 0>, <TokenType.DASH, ->, <TokenType.LITERAL_CHARACTER, 9>, <TokenType.LITERAL_CHARACTER, _>, <TokenType.CLOSED_SQUARE_BRACKET, ]>, <TokenType.CLOSED_PARENTHESIS, )>, <TokenType.STAR, *>, <TokenType.CLOSED_PARENTHESIS, )>, <TokenType.CLOSED_PARENTHESIS, )>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_PARENTHESIS, (>, <TokenType.OPEN_SQUARE_BRACKET, [>, <TokenType.LITERAL_CHARACTER, !>, <TokenType.QUESTION_MARK, ?>, <TokenType.CLOSED_SQUARE